In [ ]:
import geopandas as gpd
import xarray as xr
from rasterio.features import rasterize
from affine import Affine
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder  #For encoding categorical variables.

In [ ]:
#Create a function to rasterise the dataset.
def rasterise_polygons(var, vector_gdf, ncol_x, ncol_y, transform):

    #Create shapes object for current polygon.
    shapes = [(polygon, value) for polygon, value in zip(vector_gdf.geometry, vector_gdf[var])]

    #Rasterise the shapefile.
    return(rasterize(shapes,out_shape=(ncol_y, ncol_x),transform=transform,fill=0,dtype='float32'))

In [ ]:
#Generate the transform for the parent grid.
#Get the extent of the dataset geographically.
#This will be used to set a raster to remap the data to. 
def gen_parent_grid(vector_gdf,res):
    xmin, ymin, xmax, ymax = vector_gdf.total_bounds
    ncol_x = np.int64(np.ceil((xmax - xmin)/res)) + 1
    ncol_y = np.int64(np.ceil((ymax - ymin)/res)) + 1
    #Create the transform needed to rasterise.
    transform = Affine.translation(xmin, ymax) * Affine.scale(res, -res)
    x_coord = np.arange(xmin, xmin + (1000.0*ncol_x), step=1000.0)
    y_coord = np.flip(np.arange(ymin, ymin + (1000.0*ncol_y), step=1000.0)) #Need to flip Y_coord due to raster starting in top left
    return ncol_x, ncol_y, x_coord, y_coord, transform

In [ ]:
#Function to convert categorical variables to integers
#Use ordinal encoding.
def cat_data_to_int(vector_gdf, cat_cols):

    enc   = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    X_int = enc.fit_transform(vector_gdf[cat_cols])

    #Store the categories for future use if needed.
    category_to_int = {
        col: {cat: i for i, cat in enumerate(enc.categories_[j])}
        for j, col in enumerate(categorical_columns)
    }

    return X_int, category_to_int


In [ ]:
#Set paths to the respective shapefiles holding the soil data (carbon, parent material, bulk density).
PM1km_shp = "/gws/ssde/j25b/eds_ai/frame-fm/data/inputs/soil_parent_material_1km/data/SPMM_1km/SoilParentMateriall_V1_portal1km.shp"          #Parent material 1km
Scarb_shp = "/gws/ssde/j25b/eds_ai/frame-fm/data/inputs/model_estimates_of_topsoil_carbon/data/CS_topsoil_carbon.shp"                         #Soil Carbon
BulkD_shp = "/gws/ssde/j25b/eds_ai/frame-fm/data/inputs/model_estimates_of_topsoil_pH_and_bulk_density/data/CS_topsoil_pH_bulkDensity.shp"    #Bulk Density.

In [ ]:
#Read in the respective shapefiles.
PM1km_gdf = gpd.read_file(PM1km_shp)
Scarb_gdf = gpd.read_file(Scarb_shp)
BulkD_gdf = gpd.read_file(BulkD_shp)

In [ ]:
#Convert the categorical columns of the soil parent material dataframe.
categorical_columns                  = ['ESB_DESC','CARB_CNTNT','PMM_GRAIN','SOIL_GROUP','SOIL_TEX','SOIL_DEPTH']
PM_cat_var, PM_cat_var_dict          = cat_data_to_int(PM1km_gdf, categorical_columns)
PM1km_gdf.loc[:,categorical_columns] = PM_cat_var

In [ ]:
#We are using the parent material grid as the parent so get the coordinates for the conversion to xarray object.
grdout_ncol_x, grdout_ncol_y, grdout_x_coord, grdout_y_coord, grdout_transform = gen_parent_grid(PM1km_gdf,1000.0)

In [ ]:
#Create the initial xarray output object and regrid the parent material to the main grid.
ds_out = ds = xr.Dataset(coords={'x': grdout_x_coord, 'y': grdout_y_coord})
#Add some attributes - these will be arbitrary for now to demonstrate how it looks but will need adding to later.
ds_out.attrs['Title']  = 'FRAME_FM Soil properties 1km'
ds_out.attrs['crs']    = str(PM1km_gdf.crs)

In [ ]:
#Now add the various rasterised quantities of importance to the dataset.
#At this stage there will be no units and minimal metadata as this is just initial testing of shapefile to xarray dataset.
for f in categorical_columns:

    #Print progress.
    print('Currently processing column: '+f)
    
    #Rasterise the current column.
    curr_val_grd = rasterise_polygons(f, PM1km_gdf, grdout_ncol_x, grdout_ncol_y, grdout_transform)

    #Convert to xarray object and merge into main dataset.
    da_curr_val = xr.DataArray(curr_val_grd, coords=dict(y=grdout_y_coord,x=grdout_x_coord), name=f)
    ds_out      = xr.merge([ds_out, da_curr_val])

In [ ]:
#Now repeat for the soil carbon properties. 
soil_Carb_cols = ['CCONC_07']
for f in soil_Carb_cols:

    #Print progress.
    print('Currently processing column: '+f)
    
    #Rasterise the current column.
    curr_val_grd = rasterise_polygons(f, Scarb_gdf, grdout_ncol_x, grdout_ncol_y, grdout_transform)

    #Convert to xarray object and merge into main dataset.
    da_curr_val = xr.DataArray(curr_val_grd, coords=dict(y=grdout_y_coord,x=grdout_x_coord), name=f)
    ds_out      = xr.merge([ds_out, da_curr_val])

In [ ]:
#Now repeat for the soil carbon properties. 
soil_BD_cols = ['BULKD_07']
for f in soil_BD_cols:

    #Print progress.
    print('Currently processing column: '+f)
    
    #Rasterise the current column.
    curr_val_grd = rasterise_polygons(f, BulkD_gdf, grdout_ncol_x, grdout_ncol_y, grdout_transform)

    #Convert to xarray object and merge into main dataset.
    da_curr_val = xr.DataArray(curr_val_grd, coords=dict(y=grdout_y_coord,x=grdout_x_coord), name=f)
    ds_out      = xr.merge([ds_out, da_curr_val])

In [ ]:
ds_out

In [ ]:
ds_out['BULKD_07'].plot()

In [ ]:
BulkD_gdf['BULKD_07'].max()

In [ ]:
#Inspect the CRS of each dataset.
print(PM1km_gdf.crs)
print(Scarb_gdf.crs)
print(BulkD_gdf.crs)

In [ ]:
PM1km_gdf.head()

In [ ]:
Scarb_gdf.head()

In [ ]:
BulkD_gdf.head()

In [ ]:
#We want to get everything onto a common 1km grid and will use the soil parent material for this.
#Extract the X/Y coordinates.
#all_coords = np.round(np.vstack(PM1km_gdf.geometry.centroid.apply(lambda c: c.coords[0]).to_numpy()))
all_coords = np.round(np.vstack(Scarb_gdf.geometry.centroid.apply(lambda c: c.coords[0]).to_numpy()))

In [ ]:
all_coords

In [ ]:
#PM1km_gdf.geometry.centroid
#np.unique(np.diff(all_coords[:,1]))
np.max(np.diff(all_coords[:,1]))

In [ ]:
plt.scatter(all_coords[:,0], all_coords[:,1])
plt.show()

In [ ]:
PM1km_gdf.total_bounds

In [ ]:
Scarb_gdf.total_bounds

In [ ]:
BulkD_gdf.total_bounds

#Things to work on. 

1) Check 1km Parent material grid has extracted correct coordinates.
2) Find exactly which parent material values, carbon and bulk density variables need extracting.
3) If parent material grid is ok, map all carbon and bulk density variables to parent material grid - use demo interpolation from soil carbon sprint notebook.
4) Create one single xarray object (or one for each dataset) to map into the dataloader.
5) Turn all into python functions for current dataloader.